## **Student's Autograder**
This is a project to calculate quiz and term scores from q&a pdf and response pdf.

Whats left: 
* multi option evaluation logic
* 
* Editing pdf

Importing the required libraries 

In [1]:
import os
import pandas as pd
import numpy as np
import tabula
from bs4 import BeautifulSoup
from PyPDF2 import PdfFileReader, PdfFileWriter
from PyPDF2.generic import DecodedStreamObject, EncodedStreamObject,ArrayObject

In [2]:
#debugging for notebooks current directory for reading files later.
import os
os.getcwd()

'C:\\Users\\506761\\OneDrive - Tata Steel Limited\\Documents\\Python Scripts'

### Reading response pdf

In [3]:
df = tabula.read_pdf('POD22S1C31570014.pdf',pages='all',multiple_tables=False)

In [4]:
filename="IIT M DIPLOMA QUIZ1 EXAM QPF1.pdf"

In [5]:
df[0].tail()

,Name,SHEKHAR MOHANTY HEMANT
273,640653345424,Unanswered
274,640653345425,Unanswered
275,640653345427,Unanswered
276,640653345418,Unanswered
277,640653345419,Unanswered


### Variables initialisation

In [6]:
my_question="Question ID"
my_answer="Options Selected"
unanswered="Unanswered"
file_qid="Question Id"

In [7]:
df[0].iloc[4].values

array(['Question ID', 'Options Selected'], dtype=object)

In [8]:
df[0].head()

,Name,SHEKHAR MOHANTY HEMANT
0,QP Set,2022 Jun: IIT M DIPLOMA QUIZ1 EXAM QPF1
1,Test Center Name,iON Digital Zone iDZ Jamshedpur
2,Shift,AN
3,Roll No,POD22S1C31570014
4,Question ID,Options Selected


In [9]:
df1=df[0].replace(unanswered,np.nan,regex=True)

Students response for exam dataframe

In [10]:
myans=df1.dropna()
myans.columns=myans.iloc[4].values
myans=myans[5:]

myans[my_question]=myans[my_question].apply(int)
myans=myans.sort_values(my_question)
print(len(myans))
myans

48


,Question ID,Options Selected
139,640653345272,6406531148424
140,640653345273,6406531148426
145,640653345274,6406531148432
148,640653345275,6406531148435
146,640653345276,6406531148441
149,640653345277,6406531148444
141,640653345278,6406531148448
142,640653345279,6406531148452
151,640653345280,6406531148457
143,640653345281,6406531148458


In [11]:

# def replace_text(content, replacements = dict()):
#     lines = content.splitlines()

#     result = ""
#     in_text = False
#     for line in lines:
#         if line == "BT":
#             in_text = True

#         elif line == "ET":
#             in_text = False

#         elif in_text:
#             cmd = line[-2:]
#             if cmd.lower() == 'tj':
#                 replaced_line = line
#                 for k, v in replacements.items():
#                     replaced_line = replaced_line.replace(k, v)
#                 result += replaced_line + "\n"
#             else:
#                 result += line + "\n"
#     return result


# def process_data(object, replacements):
#     data = object.getData()
#     decoded_data = data.decode("utf-8")
#     replaced_data = replace_text(decoded_data, replacements)

#     encoded_data = replaced_data.encode("utf-8")
    
#     if object.decodedSelf is not None:
#         object.decodedSelf.setData(encoded_data)
#     else:
#         object.setData(encoded_data)




# # Provide replacements list that you need here
# replacements = { 'IITM': '<002c00510047004c004400510003002c005100560057004c00570058005700480003005200490003003700480046004b00510052004f0052004a005c000f0003003000440047005500440056000300100003002600480051005700550048>'}
# pdf = PdfFileReader("IIT M QUIZ 1 FOUNDATION DAD DIPLOMA QPD1.pdf")
# writer = PdfFileWriter()

# for page_number in range(0, pdf.getNumPages()):

#     page = pdf.getPage(page_number)
#     contents = page.getContents()

#     if isinstance(contents, DecodedStreamObject) or isinstance(contents, EncodedStreamObject):
#         process_data(contents, replacements)
#     elif len(contents) > 0:
#         for obj in contents:
#             if isinstance(obj, DecodedStreamObject) or isinstance(obj, EncodedStreamObject):
#                 streamObj = obj.getObject()
#                 process_data(streamObj, replacements)
#     page[NameObject("/Contents")] = contents.decodedSelf
#     writer.addPage(page)

# with open("result.pdf", 'wb') as out_file:
#     writer.write(out_file)

In [12]:
def gentextarr(content):
    """
    Returns an array of text block properties of pdf
    Parameter:
        content: string object of page content obtained by page.getContents().getData().decode() 
    """
    lines = content.splitlines()
    result = []
    in_text = False
    for line in lines:
        if line == "BT":
            in_text = True
            temp=[]

        elif line == "ET":
            in_text = False
            result.append(temp)
            temp=[]

        elif in_text:
            if line[-2:].lower()=="tj" or line[-2:].lower()=="tf" or line[-2:].lower()=="rg":
                temp.append(line[:])
    return result

def gen_cmap(font,pg):
    """
    Returns a character map dictionary of a pdf 'filename' page 'pg' and font 
    Parameter:
     font: font key name in meta properties of pdf.
     pg: int page number of pdf
    """
    # pdf = PdfFileReader(filename)
    # page = pdf.getPage(pg)
    page=pg
    temp=False
    cmap={}
    for i in (page.getObject()['/Resources']['/Font'][font]['/ToUnicode'].getObject().getData().decode().split('\n')):
        if "endbfrange" in i:
            temp=False
        if temp:
            cmap[i[1:5]]=i[-5:-1]
        if "beginbfrange" in i:
            temp=True
    return cmap

def font_decoder(text,cmap):
    """
    Returns decoded word from pdf text hex code. 
    """

    count=0
    word=""
    temp=""
    for i in text:
        count+=1
        if count==4:
            temp+=i
            word+=chr(int(cmap.get(temp.lower(),temp.lower()),16))
            temp=""
            count=0
        else:
            temp+=i
    return word


In [13]:
# pdf = PdfFileReader(filename)

# # for page_number in range(0, pdf.getNumPages()):
# p=False
# pdf_content=""
# for i in range(0,pdf.getNumPages()):
#     page = pdf.getPage(i)
#     contents = page.extract_text()
#     if myans[my_question].values[0] in contents:
#         sp=i
#         p=True
#     if p:
#         pdf_content+=pdf.pages[i].getContents().getData().decode()
#     if myans[my_question].values[-1] in contents:
#         ep=i
#         p=False
#         break
# if i+1<pdf.getNumPages():
#     pdf_content+=pdf.pages[i+1].getContents().getData().decode()

In [14]:
pdf = PdfFileReader(filename)

In [15]:
cmapd={}
for i in range(pdf.getNumPages()):
    for k in pdf.pages[i].getObject()['/Resources']['/Font']:
        cmap=gen_cmap(k,pdf.pages[i])
        if cmap not in cmapd.values():
            cmapd[k+str(i)]=cmap

In [33]:

sum=0
df=[[file_qid,"Correct Marks","Answers"]]
m_bool=False
marks=""
ans_bool=False
ans=""
row=[]
temp=[]
for p in range(0,pdf.getNumPages()):
    content=pdf.pages[p].getContents()
    
    if type(content)==ArrayObject:
        m=""
        for n in content:
            m+=n.getObject().getData().decode()
        content=m
    elif type(content)==EncodedStreamObject or type(content)==DecodedStreamObject:
        content=content.getData().decode()
    for i in gentextarr(content):
        text=""
        ans_bool=False
        for j in i:
            if j[-2:]=="Tf":
                cmap=gen_cmap(j.split()[0],pdf.pages[p])    
            if j=="0 0.50196 0 rg" and len(row)>1:
                ans_bool=True
            elif j[-2:].lower()=="tj":
                text=font_decoder(j[1:-3],cmap)
                if (not m_bool) and (not ans_bool):
                    # print(text)
                    if str(file_qid) in text:
                        if len(temp)>0:
                            row.append(temp)
                            df.append(row)
                        row=[]
                        temp=[]
                        if text.split()[7].isnumeric():
                            row=[text.split()[7]]
                            m_bool=True

                elif m_bool:
                    if "Correct Marks" in text:
                        # print("Marks=",text.split())
                        if len(row)==1:
                            row.append(text.split()[-1])
                        m_bool=False
                elif ans_bool:
                    if text!=" ":
                        if len(text)>=13 and text[:13].isnumeric():
                            temp.append(text[:13])
                        elif len(text.split())==3 and "to" in text.split()[1]:
                            t=text.split()
                            temp=[float(t[0]),float(t[-1])]
                        elif len(temp)==0:
                            temp.append(text)
if len(row)==3:
    df.append(row)

In [34]:
print(len(df))
df[10]

278


['640653345137', '2', ['1']]

In [35]:
df2=pd.DataFrame(df[1:],columns=df[0])
df2[file_qid]=df2[file_qid].apply(int)
df2=df2.sort_values([file_qid])
df2.head()

,Question Id,Correct Marks,Answers
0,640653345126,0,[6406531148006]
1,640653345127,2,[9]
2,640653345128,2,[6]
3,640653345129,2,[4]
4,640653345130,4,[4]


In [36]:
df2=df2[df2[file_qid].isin(myans[my_question].values)].reset_index().drop("index",axis=1)
df2.head()

,Question Id,Correct Marks,Answers
0,640653345272,0,[6406531148424]
1,640653345273,3,[6406531148426]
2,640653345274,2,[6406531148430]
3,640653345275,5,[6406531148434]
4,640653345276,2,[6406531148441]


In [37]:
df2["My answer"]=myans.reset_index()[my_answer]
df2["My qid"]=myans.reset_index()[my_question]
print(len(df2))
df2

48


,Question Id,Correct Marks,Answers,My answer,My qid
0,640653345272,0,[6406531148424],6406531148424,640653345272
1,640653345273,3,[6406531148426],6406531148426,640653345273
2,640653345274,2,[6406531148430],6406531148432,640653345274
3,640653345275,5,[6406531148434],6406531148435,640653345275
4,640653345276,2,[6406531148441],6406531148441,640653345276
5,640653345277,5,[6406531148444],6406531148444,640653345277
6,640653345278,3,[6406531148448],6406531148448,640653345278
7,640653345279,3,[6406531148453],6406531148452,640653345279
8,640653345280,5,[6406531148457],6406531148457,640653345280
9,640653345281,3,[6406531148459],6406531148458,640653345281


In [153]:
def evaluate(df2):
    df2['Score'] = df2.apply(lambda x: float(x['Correct Marks']) if len(x["My answer"])==13 and x['My answer'] in x["Answers"] else x["Score"], axis=1)
    df2['Score'] = df2.apply(lambda x: (len(set(x["My answer"].split(",")).intersection(set(x["Answers"])))/len(x["Answers"]))*float(x['Correct Marks']) if type(x["Answers"][0])==str and len(x["Answers"][0])==13 and len(x)>1 and len(set(x["My answer"].split(",")).intersection(set(x["Answers"])))>0 else x["Score"], axis=1)
    df2['Score'] = df2.apply(lambda x: float(x['Correct Marks']) if len(x["My answer"])<13 and len(x['Answers'])>1 and float(x['My answer']) >= float(x["Answers"][0]) and float(x['My answer']) <= float(x["Answers"][1]) else x["Score"], axis=1)
    df2['Score'] = df2.apply(lambda x: float(x['Correct Marks']) if len(x["My answer"])<13 and len(x['Answers'])==1 and float(x['My answer']) == float(x["Answers"][0]) else x["Score"], axis=1)
    return df2

In [154]:
df2['Score']=np.zeros(df2.shape[0])
df2=evaluate(df2)
df2

,Question Id,Correct Marks,Answers,My answer,My qid,Score
0,640653345272,0,[6406531148424],6406531148424,640653345272,0.0
1,640653345273,3,[6406531148426],6406531148426,640653345273,3.0
2,640653345274,2,[6406531148430],6406531148432,640653345274,0.0
3,640653345275,5,[6406531148434],6406531148435,640653345275,0.0
4,640653345276,2,[6406531148441],6406531148441,640653345276,2.0
5,640653345277,5,[6406531148444],6406531148444,640653345277,5.0
6,640653345278,3,[6406531148448],6406531148448,640653345278,3.0
7,640653345279,3,[6406531148453],6406531148452,640653345279,0.0
8,640653345280,5,[6406531148457],6406531148457,640653345280,5.0
9,640653345281,3,[6406531148459],6406531148458,640653345281,0.0


## Final score

In [53]:
df2["Score"].apply(float).sum()

81.0

In [51]:
{int(j) for j in x['My answer'][0].split(",")}.intersection(set(x["Answers"][0]))

{640653345406}

In [26]:
x=pd.DataFrame({'My answer':"640653345406","Answers":[[640653345406,640653345405]],"Score":2})

In [27]:
{int(i) for i in x['My answer'][0].split(",")}

{640653345406}

In [28]:
map(int,x["My answer"][0])

In [95]:
s_index=df2[df2["Correct Marks"]=="0"].index
s_index.shape

(3,)

In [155]:
subjects=[]
for i in range(len(s_index)):
    if i==len(s_index)-1:
        subjects.append(df2.iloc[s_index[i]:])
    else:
        subjects.append(df2.iloc[s_index[i]:s_index[i+1]])
print(subjects)

[     Question Id Correct Marks                         Answers  \
0   640653345272             0                 [6406531148424]   
1   640653345273             3                 [6406531148426]   
2   640653345274             2                 [6406531148430]   
3   640653345275             5                 [6406531148434]   
4   640653345276             2                 [6406531148441]   
5   640653345277             5                 [6406531148444]   
6   640653345278             3                 [6406531148448]   
7   640653345279             3                 [6406531148453]   
8   640653345280             5                 [6406531148457]   
9   640653345281             3                 [6406531148459]   
10  640653345283             5  [6406531148466, 6406531148468]   
11  640653345284             2  [6406531148471, 6406531148473]   
12  640653345285             3                 [6406531148475]   
13  640653345286             2                 [6406531148478]   
14  64065

In [156]:
subjects[0]

,Question Id,Correct Marks,Answers,My answer,My qid,Score
0,640653345272,0,[6406531148424],6406531148424,640653345272,0.0
1,640653345273,3,[6406531148426],6406531148426,640653345273,3.0
2,640653345274,2,[6406531148430],6406531148432,640653345274,0.0
3,640653345275,5,[6406531148434],6406531148435,640653345275,0.0
4,640653345276,2,[6406531148441],6406531148441,640653345276,2.0
5,640653345277,5,[6406531148444],6406531148444,640653345277,5.0
6,640653345278,3,[6406531148448],6406531148448,640653345278,3.0
7,640653345279,3,[6406531148453],6406531148452,640653345279,0.0
8,640653345280,5,[6406531148457],6406531148457,640653345280,5.0
9,640653345281,3,[6406531148459],6406531148458,640653345281,0.0


In [157]:
reports=[]
for i in subjects:
    print(i["Correct Marks"].apply(float).values)
    reports.append((i["Correct Marks"].apply(float).values.sum(),i["Score"].sum()))

[0. 3. 2. 5. 2. 5. 3. 3. 5. 3. 5. 2. 3. 2. 5.]
[0.  4.5 2.  2.  2.  3.  3.  3.  3.  3.  3.  4.5 4.5 4.5 3.  3.  2. ]
[0. 1. 1. 1. 1. 2. 2. 1. 2. 1. 1. 1. 1. 2. 1. 1.]


In [160]:
for i in reports:
    print(i,"score",i[1]/i[0]*100)

(48.0, 30.0) score 62.5
(50.0, 38.0) score 76.0
(19.0, 13.0) score 68.42105263157895
